# Scraping the website of the "Schaubühne"

[schaubuehne website](https://www.schaubuehne.de/de/start/index.html)

### Importing html with Beautiful Soup

In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
sbühne_http_response = requests.get ("https://www.schaubuehne.de/de/premieren/2024-25.html")
sbühne_http_response

<Response [200]>

In [6]:
sbühne_html = sbühne_http_response.text

In [7]:
print(sbühne_html)

<!DOCTYPE html>
<html lang="de">
<head>
<base href="https://www.schaubuehne.de/de/" />
<meta name="google-site-verification" content="eEc9v1XtKjDjOJN-5sBC5AYEq2QYtiXQ1_lXduY-wQQ">
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>Schaubühne – Spielplan&nbsp;–&nbsp;Premieren</title>

<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" />
<meta name="description" content="Die Schaubühne ist ein Theater in Berlin. Die Website informiert über den Spielplan, Schauspieler, Regisseure, Gastspiele, Aktuelles. Tickets sind online im Webshop buchbar." />
<meta name="keywords" content="Schaubühne, Theater, Berlin, Lehniner Platz" />
<meta name="robots" content="index,follow" />
<meta name="googlebot"  content="index,follow" />


<link rel="stylesheet" type="text/css" href="./../assets/js/cookieconsent2/cookieconsent.min.css" />


<!-- Facebook -->
<meta property="og:url" content="https://www.schaubuehne.de/de/premieren/2024-25.html"/>
<met

In [8]:
soup = BeautifulSoup(sbühne_html)
type(soup)

bs4.BeautifulSoup

### From the overview page I am scraping the titles

In [9]:
links = soup.select('div.pe-lg-5 h2.mb-0 a.no-underline')

In [10]:
links[0].get('title')

'Glaube, Geld, Krieg und Liebe'

In [11]:
titles = [
    f"{link.get('title')}" 
    for link in links
    ]
print(titles)

['Glaube, Geld, Krieg und Liebe', 'Die Entführung der Amygdala', 'Angriffe auf Anne', 'change', 'Replay']


#### Scraping the links

In [12]:
links[0].get('href')

'./produktionen/glaube-geld-krieg-und-liebe.html?m=572'

In [13]:
list_links = [
    f"https://www.schaubuehne.de/de{link.get('href')}" 
    for link in links
    ]
print(list_links)

['https://www.schaubuehne.de/de./produktionen/glaube-geld-krieg-und-liebe.html?m=572', 'https://www.schaubuehne.de/de./produktionen/die-entfuehrung-der-amygdala.html?m=572', 'https://www.schaubuehne.de/de./produktionen/angriffe-auf-anne.html?m=572', 'https://www.schaubuehne.de/de./produktionen/change.html?m=572', 'https://www.schaubuehne.de/de./produktionen/index-4.html?m=572']


#### Scraping the description

In [14]:
def get_description(url):
    http_response = requests.get (url)
    html = http_response.text
    soup = BeautifulSoup(html)
  
    main_container = soup.find('div', class_='col col-24 col-lg-16 col-xl-18')
    description_container = main_container.find('div', class_='my-3')

    description_paragraphs = description_container.find_all('p')
    description = '\n\n'.join(p.get_text() for p in description_paragraphs)

    return description




In [15]:
descriptions = []
for link in list_links:
    description = get_description(link)
    print(description)
    descriptions.append(description)
    print("----")

Robert Lepage entwickelt ein neues Stück mit Spieler_innen aus dem Ensemble der Schaubühne. Zu Beginn der Proben gab es keinen Text, keine Geschichte, keine Figuren, nur einen Gegenstand: ein Kartenspiel. Die vier Kartenfarben Herz, Kreuz, Pik und Karo ordnet er der Liebe, dem Glauben, dem Krieg und dem Geld zu. Aus den Karten, ihren Farbfamilien, Figuren und Zahlen entstanden in Improvisationen ganze Welten, unterschiedlichste Figuren und vier miteinander verwobene Handlungsstränge, die sich über acht Jahrzehnte deutscher Geschichte erstrecken. Sie erzählen von Liebe, vom Suchen nach Glück und der Versuchung durch den Teufel, von Hoffnung, Schicksal und Trauma. Kriege markieren immer wieder eine Zäsur, ein Ende und den Neubeginn einer weiteren Geschichte: Kurz nach dem Zweiten Weltkrieg wird ein Baby in einem Nonnenkloster abgegeben und wächst dort auf, um, kaum erwachsen, die junge Bundesrepublik in Richtung Paris zu verlassen. Als sie selbst Zwillinge bekommt, prophezeit ihr eine Ta

In [16]:
len(descriptions)

5

In [17]:
len(list_links)

5

#### Scraping the director

In [18]:
def get_director(url):
    http_response = requests.get (url)
    html = http_response.text
    soup = BeautifulSoup(html)

    director_containers = soup.find_all('div', class_=['my-3', 'my-3 mb-2 fs-4 lh-lg'])
    
    for container in director_containers:
        director_span = container.find('span', string=lambda text: 'Regie' in text)
        if director_span:
            director = director_span.find_next('a').get_text()
            return director
       
    print(f"Director not found in {url}")
    return None

    


In [19]:
directors = []
for link in list_links:
    director = get_director(link)
    print(director)
    directors.append(director)
    print("----")

Robert Lepage
----
Anika  Stauch
----
Lilja Rupprecht
----
Thomas Ostermeier
----
Yael Ronen
----


In [20]:
len(directors)

5

#### Scraping the date

In [21]:

def get_date(url):
    http_response = requests.get (url)
    html = http_response.text
    soup = BeautifulSoup(html)

    date_divs = soup.find_all('div', class_='my-3 fs-4')

    for div in date_divs:
       
        p_tag = div.find('p')
        if p_tag and 'Premiere ist am' in p_tag.text:
            date_text = p_tag.text.strip().replace('Premiere ist am', '').strip()
            return date_text
 
    print(f"Date not found in {url}")
    return None


In [22]:
dates = []
for link in list_links:
    date = get_date(link)
    print(date)
    dates.append(date)
    print("----")

29. September 2024
----
14. Oktober 2024
----
30. Oktober 2024
----
30. November 2024
----
14. Dezember 2024
----


In [23]:
len(dates)

5

#### Making a dataframe in pandas

In [24]:
sb_24 = pd.DataFrame()

sb_24["link"] = list_links
sb_24["title"] = titles
sb_24["director"] = directors
sb_24["premiere_date"] = dates
sb_24["description"] = descriptions

In [25]:
sb_24

,link,title,director,premiere_date,description
0,https://www.schaubuehne.de/de./produktionen/gl...,"Glaube, Geld, Krieg und Liebe",Robert Lepage,29. September 2024,Robert Lepage entwickelt ein neues Stück mit S...
1,https://www.schaubuehne.de/de./produktionen/di...,Die Entführung der Amygdala,Anika Stauch,14. Oktober 2024,Sie hat alles im Griff. Den vollzeitfordernden...
2,https://www.schaubuehne.de/de./produktionen/an...,Angriffe auf Anne,Lilja Rupprecht,30. Oktober 2024,Anne. Anya. Annie. Anny. Annushka. Internation...
3,https://www.schaubuehne.de/de./produktionen/ch...,change,Thomas Ostermeier,30. November 2024,"Berlin, am Morgen, eine Küche. Nina und Mark, ..."
4,https://www.schaubuehne.de/de./produktionen/in...,Replay,Yael Ronen,14. Dezember 2024,Sowohl die Erd- wie auch die Menschheitsgeschi...


In [28]:
month_map = {
    'Januar': '01',
    'Februar': '02',
    'März': '03',
    'April': '04',
    'Mai': '05',
    'Juni': '06',
    'Juli': '07',
    'August': '08',
    'September': '09',
    'Oktober': '10',
    'November': '11',
    'Dezember': '12'
}

sb_24['premiere_date'] = sb_24['premiere_date'].replace(month_map, regex=True)

In [33]:
def convert_date(date_str):
    date_obj = datetime.strptime(date_str, '%d. %m %Y')
    return date_obj.strftime('%d/%m/%Y')

sb_24['premiere_date'] = sb_24['premiere_date'].apply(convert_date)

In [34]:
sb_24['premiere_date']

0    29/09/2024
1    14/10/2024
2    30/10/2024
3    30/11/2024
4    14/12/2024
Name: premiere_date, dtype: object

In [35]:
sb_24.to_csv("sb_24.csv")